In [1]:
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F
import csv 
import pandas as pd
import glob
from scipy.io.wavfile import read
import wave, os, glob
import numpy as np
import sys
import librosa
import pickle


In [2]:
"""
data = pd.read_csv("../DCASE dataset/meta.txt", sep="\t" , header = None)
print(len(data))
#data.columns = ["a", "b", "c"]
label_map = {'beach':0, 'bus':1, 'cafe/restaurant':2, 'car':3, 'city_center':4, 'forest_path':5, 'grocery_store':6, 
            'home':7, 'library':8, 'metro_station':9, 'office':10,'park':11, 'residential_area':12, 'train':13, 'tram':14}
print("labeling done")
"""

'\ndata = pd.read_csv("../DCASE dataset/meta.txt", sep="\t" , header = None)\nprint(len(data))\n#data.columns = ["a", "b", "c"]\nlabel_map = {\'beach\':0, \'bus\':1, \'cafe/restaurant\':2, \'car\':3, \'city_center\':4, \'forest_path\':5, \'grocery_store\':6, \n            \'home\':7, \'library\':8, \'metro_station\':9, \'office\':10,\'park\':11, \'residential_area\':12, \'train\':13, \'tram\':14}\nprint("labeling done")\n'

In [3]:
#print(data)
import torch.optim as optim
from sklearn.utils import shuffle
#new_data = shuffle(data)
#print(data)
batch_size=512

In [4]:
#path2="/home/nitin/audio_project/myworks/DCASE dataset"
path = "./"

In [5]:
"""
wavlabel=[]
wavpath=[]
for row in new_data[0]:
    #print(row)
    wavpath.append(row)
for row1 in new_data[1]:
    #print(row1)
    wavlabel.append(row1)
print(len(wavpath))        
"""

'\nwavlabel=[]\nwavpath=[]\nfor row in new_data[0]:\n    #print(row)\n    wavpath.append(row)\nfor row1 in new_data[1]:\n    #print(row1)\n    wavlabel.append(row1)\nprint(len(wavpath))        \n'

In [6]:
"""
print(len(wavpath))
train=[]
for i in range(len(wavpath)):
    #data1[1][i]
    wavfil=wavpath[i]
    #print(wavfil)
    label=wavlabel[i]
    #print(label)
    #print([label_map[label]])
    filename=glob.glob(os.path.join(path, wavfil))
    y, sr = librosa.load((os.path.join(path, wavfil)))
    x=librosa.feature.melspectrogram(y=y, sr=sr,n_mels=60)
    #print(x.shape)
    if(i%100==0):
        print(i)
    train.append(x)
    #print(i)
print("labeled")
"""

'\nprint(len(wavpath))\ntrain=[]\nfor i in range(len(wavpath)):\n    #data1[1][i]\n    wavfil=wavpath[i]\n    #print(wavfil)\n    label=wavlabel[i]\n    #print(label)\n    #print([label_map[label]])\n    filename=glob.glob(os.path.join(path, wavfil))\n    y, sr = librosa.load((os.path.join(path, wavfil)))\n    x=librosa.feature.melspectrogram(y=y, sr=sr,n_mels=60)\n    #print(x.shape)\n    if(i%100==0):\n        print(i)\n    train.append(x)\n    #print(i)\nprint("labeled")\n'

In [7]:
'''
print(len(wavpath))
print(len(train))
labell=[]
for i in range(len(wavlabel)):
    #print(wavlabel[i])
    #wavfil=data1[0][i]
    #print(wavfil)
    label2=wavlabel[i]
    seq=np.array(label_map[label2])
    x=np.zeros(15)
    x[seq]=1
    y=np.reshape(x,(1,15))
    labell.append(y)
print(len(labell))
print(len(train))
'''

'\nprint(len(wavpath))\nprint(len(train))\nlabell=[]\nfor i in range(len(wavlabel)):\n    #print(wavlabel[i])\n    #wavfil=data1[0][i]\n    #print(wavfil)\n    label2=wavlabel[i]\n    seq=np.array(label_map[label2])\n    x=np.zeros(15)\n    x[seq]=1\n    y=np.reshape(x,(1,15))\n    labell.append(y)\nprint(len(labell))\nprint(len(train))\n'

In [8]:
"""
with open('./60-mel_train.pkl', 'wb') as f:
    pickle.dump(train, f)
"""    

"\nwith open('./60-mel_train.pkl', 'wb') as f:\n    pickle.dump(train, f)\n"

In [9]:
"""
with open('./60-mel_train_label.pkl', 'wb') as f:
    pickle.dump(labell, f)
"""    

"\nwith open('./60-mel_train_label.pkl', 'wb') as f:\n    pickle.dump(labell, f)\n"

In [10]:
with open('./60-mel_train.pkl', 'rb') as f:
    train = pickle.load(f)
with open('./60-mel_train_label.pkl', 'rb') as f:
    labell = pickle.load(f)    

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(25860, 10000)
        self.fc2 = nn.Linear(10000, 1000)
        self.fc3 = nn.Linear(1000, 15)
        self.drop= nn.Dropout(p=0.2)
        self.sig = nn.Sigmoid()
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = F.relu(self.fc2(x))
        x = self.drop(x) 
        x = self.sig(self.fc3(x))
        #x = self.sig(x)
        return x

In [12]:
net = Net().cuda()
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [13]:
nput=train[0]
print(nput.shape)
inputss=np.reshape(nput,[1,60*431])
labels=labell[0]
#print("label---",labels)
inputs = Variable(torch.FloatTensor(torch.from_numpy(inputss).float())).cuda()
labels = Variable(torch.FloatTensor(torch.from_numpy(labels).float())).cuda()
optimizer.zero_grad()
outputs = net(inputs)
loss = criterion(outputs, labels)
loss.backward()
optimizer.step()
print(loss)


(60, 431)
Variable containing:
 0.6906
[torch.cuda.FloatTensor of size 1 (GPU 0)]



In [14]:
net.train()
for epoch in range(500):  # loop over the dataset multiple times
    for i in range(0,len(train), batch_size):
        optimizer.zero_grad()
        #print(i)
        x_batch = train[i:i+batch_size]
        x = np.reshape(x_batch,[-1,60*431])
        y_batch = labell[i:i+batch_size]
        y=np.array(y_batch)
        #print(len(x_batch))
        #print(len(y_batch))
        inputs = Variable(torch.FloatTensor(torch.from_numpy(x).float())).cuda()
        labels = Variable(torch.FloatTensor(torch.from_numpy(y).float())).cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        print(loss,epoch)
        
print('Finished Training')


/home/nitin/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1189: UserWarning: Using a target size (torch.Size([512, 1, 15])) that is different to the input size (torch.Size([512, 15])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Variable containing:
 0.6366
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 0
Variable containing:
 0.5587
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 0
Variable containing:
 0.5626
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 0
Variable containing:
 0.5437
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 0
Variable containing:
 0.5220
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 0
Variable containing:
 0.5333
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 0
Variable containing:
 0.5188
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 0
Variable containing:
 0.5266
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 0
Variable containing:
 0.5080
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 0


/home/nitin/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1189: UserWarning: Using a target size (torch.Size([72, 1, 15])) that is different to the input size (torch.Size([72, 15])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Variable containing:
 0.4877
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 0
Variable containing:
 0.4912
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 1
Variable containing:
 0.4892
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 1
Variable containing:
 0.4781
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 1
Variable containing:
 0.4996
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 1
Variable containing:
 0.4709
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 1
Variable containing:
 0.4815
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 1
Variable containing:
 0.4921
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 1
Variable containing:
 0.4843
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 1
Variable containing:
 0.4813
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 1
Variable containing:
 0.4573
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 1
Variable containing:
 0.4481
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 2
Variable containing:
 0.4686
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 2
Variable containing:
 0.4

Variable containing:
 0.2795
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 11
Variable containing:
 0.2398
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 11
Variable containing:
 0.2783
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 12
Variable containing:
 0.2751
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 12
Variable containing:
 0.2741
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 12
Variable containing:
 0.2827
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 12
Variable containing:
 0.2658
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 12
Variable containing:
 0.2568
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 12
Variable containing:
 0.2723
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 12
Variable containing:
 0.2892
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 12
Variable containing:
 0.2781
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 12
Variable containing:
 0.2210
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 12
Variable containing:
 0.2563
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 13
Variable con

...

Variable containing:
1.00000e-02 *
  1.6835
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 460
Variable containing:
1.00000e-02 *
  1.4613
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 460
Variable containing:
1.00000e-02 *
  1.2357
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 460
Variable containing:
1.00000e-02 *
  1.4734
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 460
Variable containing:
1.00000e-02 *
  1.2682
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 460
Variable containing:
1.00000e-02 *
  1.3482
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 460
Variable containing:
1.00000e-02 *
  1.1712
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 460
Variable containing:
1.00000e-02 *
  1.1427
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 460
Variable containing:
1.00000e-02 *
  1.1855
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 460
Variable containing:
1.00000e-03 *
  4.1018
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 460
Variable containing:
1.00000e-02 *
  1.6655
[torch.cuda.FloatTensor of size 1 (G

Variable containing:
1.00000e-02 *
  1.6620
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 469
Variable containing:
1.00000e-02 *
  1.9356
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 469
Variable containing:
1.00000e-02 *
  1.1061
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 469
Variable containing:
1.00000e-02 *
  1.0633
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 469
Variable containing:
1.00000e-02 *
  1.2161
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 469
Variable containing:
1.00000e-02 *
  1.0359
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 469
Variable containing:
1.00000e-02 *
  1.0886
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 469
Variable containing:
1.00000e-02 *
  1.2322
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 469
Variable containing:
1.00000e-02 *
  1.2453
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 469
Variable containing:
1.00000e-03 *
  4.9092
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 469
Variable containing:
1.00000e-02 *
  1.6669
[torch.cuda.FloatTensor of size 1 (G

Variable containing:
1.00000e-02 *
  1.4914
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 478
Variable containing:
1.00000e-02 *
  1.3110
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 478
Variable containing:
1.00000e-02 *
  1.0469
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 478
Variable containing:
1.00000e-03 *
  9.3967
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 478
Variable containing:
1.00000e-02 *
  1.1818
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 478
Variable containing:
1.00000e-03 *
  9.8178
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 478
Variable containing:
1.00000e-02 *
  1.0327
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 478
Variable containing:
1.00000e-02 *
  1.4552
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 478
Variable containing:
1.00000e-02 *
  1.4282
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 478
Variable containing:
1.00000e-03 *
  3.5363
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 478
Variable containing:
1.00000e-02 *
  1.5705
[torch.cuda.FloatTensor of size 1 (G

Variable containing:
1.00000e-02 *
  1.6981
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 487
Variable containing:
1.00000e-02 *
  1.5746
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 487
Variable containing:
1.00000e-02 *
  1.1511
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 487
Variable containing:
1.00000e-02 *
  1.1756
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 487
Variable containing:
1.00000e-02 *
  1.2366
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 487
Variable containing:
1.00000e-02 *
  1.1153
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 487
Variable containing:
1.00000e-02 *
  1.0840
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 487
Variable containing:
1.00000e-02 *
  1.2308
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 487
Variable containing:
1.00000e-02 *
  1.2606
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 487
Variable containing:
1.00000e-02 *
  1.0215
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 487
Variable containing:
1.00000e-02 *
  3.5914
[torch.cuda.FloatTensor of size 1 (G

Variable containing:
1.00000e-02 *
  1.3991
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 496
Variable containing:
1.00000e-02 *
  1.7225
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 496
Variable containing:
1.00000e-02 *
  2.1889
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 496
Variable containing:
1.00000e-02 *
  1.5452
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 496
Variable containing:
1.00000e-02 *
  1.4384
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 496
Variable containing:
1.00000e-02 *
  1.2326
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 496
Variable containing:
1.00000e-02 *
  1.6901
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 496
Variable containing:
1.00000e-02 *
  1.8017
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 496
Variable containing:
1.00000e-02 *
  1.7090
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 496
Variable containing:
1.00000e-03 *
  5.1410
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 496
Variable containing:
1.00000e-02 *
  1.5984
[torch.cuda.FloatTensor of size 1 (G

In [15]:
"""
test_data = pd.read_csv("../DCASE dataset/test/meta.txt", sep="\t" , header = None)
print(len(test_data))
test = []
#path2="/home/nitin/audio_project/myworks/DCASE dataset"
path = "./test/"
"""

'\ntest_data = pd.read_csv("../DCASE dataset/test/meta.txt", sep="\t" , header = None)\nprint(len(test_data))\ntest = []\n#path2="/home/nitin/audio_project/myworks/DCASE dataset"\npath = "./test/"\n'

In [16]:
"""
for i in range(len(test_data)):
    wavfil=test_data[0][i]
    #print(wavfil)
    label=test_data[1][i]
    #print(label)
    #print([label_map[label]])
    filename=glob.glob(os.path.join(path, wavfil))
    y, sr = librosa.load((os.path.join(path, wavfil)))
    x=librosa.feature.melspectrogram(y=y, sr=sr,n_mels=60)
    if(i%100==0):
        print(i)
    test.append(x)
    #print(x.shape)
print("labeled")
"""

'\nfor i in range(len(test_data)):\n    wavfil=test_data[0][i]\n    #print(wavfil)\n    label=test_data[1][i]\n    #print(label)\n    #print([label_map[label]])\n    filename=glob.glob(os.path.join(path, wavfil))\n    y, sr = librosa.load((os.path.join(path, wavfil)))\n    x=librosa.feature.melspectrogram(y=y, sr=sr,n_mels=60)\n    if(i%100==0):\n        print(i)\n    test.append(x)\n    #print(x.shape)\nprint("labeled")\n'

In [17]:
#print(test[0].shape)

In [18]:
""""
test_label=[]
for i in range(len(test_data)):
    wavfil=test_data[0][i]
    #print(wavfil)
    label=test_data[1][i]
    seq=np.array(label_map[label])
    x=np.zeros(15)
    x[seq]=1
    y=np.reshape(x,(1,15))
    test_label.append(y)
#print(labell)    
"""

'"\ntest_label=[]\nfor i in range(len(test_data)):\n    wavfil=test_data[0][i]\n    #print(wavfil)\n    label=test_data[1][i]\n    seq=np.array(label_map[label])\n    x=np.zeros(15)\n    x[seq]=1\n    y=np.reshape(x,(1,15))\n    test_label.append(y)\n#print(labell)    \n'

In [19]:
""""
with open('./60-mel_test.pkl', 'wb') as f:
    pickle.dump(test, f)
"""    

'"\nwith open(\'./60-mel_test.pkl\', \'wb\') as f:\n    pickle.dump(test, f)\n'

In [20]:
"""
with open('./60-mel_train_label.pkl', 'wb') as f:
    pickle.dump(test_l, f)
"""    

"\nwith open('./60-mel_train_label.pkl', 'wb') as f:\n    pickle.dump(test_l, f)\n"

In [21]:
with open('./60-mel_test.pkl', 'rb') as f:
    test = pickle.load(f)
with open('./60-mel_test_label.pkl', 'rb') as f:
    test_label = pickle.load(f)    

In [22]:
correct=0
for i in range(1):
    x_batch = train[i]
    x = np.reshape(x_batch,[-1,60*431])
    y_batch = labell[i]
    #print(y_batch)
    y=np.array(y_batch)
    inputs = Variable(torch.FloatTensor(torch.from_numpy(x).float())).cuda()
    #labels = Variable(torch.FloatTensor(torch.from_numpy(y).float())).cuda()
    outputs = net(inputs)
    #print(outputs)
    values, indices = torch.max(outputs, 0)
    #print(values)
    #print(indices)
    #print(i)


In [23]:
net.eval()
correct=0
for i in range(len(train)):
    x_batch = train[i]
    x = np.reshape(x_batch,[-1,60*431])
    y_batch = labell[i]
    #print(y_batch)
    y=np.array(y_batch)
    inputs = Variable(torch.FloatTensor(torch.from_numpy(x).float())).cuda()
    labels = Variable(torch.FloatTensor(torch.from_numpy(y).float())).cuda()
    outputs = net(inputs)
    m = nn.Softmax(dim=1)
    #print(outputs)
    output1 = m(outputs)
    #print(output1)
    values, indices = torch.max(labels.data[0], 0)
    values1, indices1 = torch.max(outputs.data[0], 0)
    x1=indices.cpu().numpy()
    x2=indices1.cpu().numpy()
    if(x1==x2):
        correct+=1
        #print("hello")


In [24]:
print(correct)

4619


In [25]:
print(len(train))

4680


In [26]:
print(correct/len(train))

0.986965811965812


In [27]:
net.eval()
test_correct=0
for i in range(len(test)):
    x_batch = test[i]
    #print(x_batch.shape)
    x = np.reshape(x_batch,[-1,60*431])
    y_batch = test_label[i]
    #print(y_batch)
    y=np.array(y_batch)
    inputs = Variable(torch.FloatTensor(torch.from_numpy(x).float())).cuda()
    labels = Variable(torch.FloatTensor(torch.from_numpy(y).float())).cuda()
    outputs = net(inputs)
    m = nn.Softmax(dim=1)
    #print(outputs)
    output1 = m(outputs)
    #print(output1)
    values, indices = torch.max(labels.data[0], 0)
    values1, indices1 = torch.max(outputs.data[0], 0)
    x1=indices.cpu().numpy()
    x2=indices1.cpu().numpy()
    if(x1==x2):
        test_correct+=1
        #print("hello")

In [28]:
print(test_correct)
print(len(test))
acc=test_correct/(len(test))

589
1620


In [29]:
print(acc)

0.36358024691358026
